<a href="https://colab.research.google.com/github/MarlonRF/CPGIF_Colab/blob/master/Raspagem_de_dados_do_Google_Scholar_Simples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Avisos, sugestões de uso

* Faça upload de arquivo Excel com nome "lista de nomes.xlsx". Atenção ao formato .xlsx. Se realizar o upload de arquivo com outro nome ou extensão, vai ter que alterar o script.

* Faça um teste com uma lista pequena, no máximo 5 nomes. 

* Se a lista de nomes for superior a 50 nomes, recomendo quebrar a lista em várias outras e executar cada uma separadamente. Buscas continuas no Google Scholar, com o mesmo endereço de IP, vão levantar alerta sobre o uso de bots. O seu acesso será bloqueado por uns dias (ainda vai conseguir acessar via Browser, mas com CAPTCHA). Dê um intervalo de 1 hora entre a execução das listas.

* O tempo de excecusão pode variar. Quanto maior a lista, maior a variação. O teste com 5 nomes, não levou mais que 5 minutos.

* Vou aperfeiçoar o script ao longo do tempo, mas se houver algum bloqueio ou mudanças no Google Scholar, o script pode deixar de funcionar. 

* Nem todos os nomes serão encontrados pelos mais variados motivos, mesmo que existam no Google Scholar (falso negativo). A dificuldade mais comum serão os casos em que o pesquisador usar um bem diferente no nome utilizado na busca, por exemplo: Cesare Mansueto Giulio Lattes -> César Lattes 

* Alguns nomes que não fazem parte da lista serão adicionados aos resultados (falso positivo). Próxima versão do script implemento uma solução.

* Vou 
---
# 2. Instruções

A lista de nome é a mais simples possível. Preferêncialmene nome completo. 
<img src='https://drive.google.com/uc?id=1kUP8ucC2FcKidYPq6nMJN4rjZabCvpUt'></img>



## 2.1 Upload do arquivo Excel

Clicar no 4º ícone à esquerda (Arquivos) e depois no 1º icone dos 3 que irão surgir "Fazer Upload para armaza..."
<img src='https://drive.google.com/uc?id=1sMANy4ZCFue1iSpPP0FrclXAvAMMR6Eo'></img>



## 2.2. Executar o script
Clique no ícone da célular onde está escrito o script
<img src='https://drive.google.com/uc?id=1PxedJ08n7lUgbc3JdjF0hakOjujOeuhE'></img>



## 2.3 Download do arquivo.
 Dados serão salvos no mesmo arquivo, mas em outra planilha. Só baixar
 <img src='https://drive.google.com/uc?id=1AZWAkYDESKBU1sr1vj4R5HFnBK7aXvDF'></img>
 



 Resultado deve esse.
 <img src='https://drive.google.com/uc?id=1MwPQypO9mR23e85QH62j5_4dXBEGEZ6R'></img>




# Nova seção



---
# SCRIPT!
É só executar. Se você for bloqueado pelo Google, vai receber mensagem de erro. O jeito é esperar uns dias para ser desbloqueado.



In [ ]:

# Suprime todo Output dessa celula 
!pip install scholarly # instala biblioteca no COLAB para execução
!pip install scraperapi-sdk

# Bibliotecas
from scholarly import scholarly, ProxyGenerator # Faz o meio de campo com o BeautfulSoup para obter os dados no GoogleScholar 
import pandas as pd # Dataframes
import numpy as np # Cálculos numéricos
from scraper_api import ScraperAPIClient # Alternativa para utilizar outros Proxies 
import random # Para utilizar aleatórios
import pickle # salva dados do Python
import matplotlib.pyplot as plt # Gráficos

# Funções

# Quebra o nome completo e compõe novo nome: 1º e último
def quebrarNome(string):
  n = string.split()
  ultimo = len(n)-1
  novo_n = (n[0]+' '+n[ultimo])
  return novo_n

# usa proxy para variar IP. Google bloqueia bots

# busca e verifica a existencia nomes no Google Acadêmico. 
# Retorna a lista dos nomes encontrados na forma encontrada
# Exemplo: Cesare Mansueto Giulio Lattes -> César Lattes

# Histograma com linha da média e da mediana
def histograma(x, bins=40,label_x='',label_y='',titulo='', subtitulo=''):
  """Gráfico Histograma
  Parameters
  ----------
  x : dados númericos: série do Pandas, vetor do numpy
  xbins: nº de bins
  label_x: rótulo o eixo X
  label_y: rótulo do eixo y
  titulo: Título do histograma
  subtitulo: Subtítulo do histograma

  Returns
  -------
  Imagem :
      retorna histograma
  """
  x = x[~np.isnan(x)] # descarta os valores NA
  result = plt.hist(x, bins=40, 
                    color='blue', 
                    edgecolor='darkblue', 
                    alpha=0.2)
  plt.axvline(x.mean(), # Linha da média
              color='darkblue', 
              linestyle='dashed', 
              linewidth=1
              )
  plt.axvline(np.median(x), #linha da mediana
              color='darkred', 
              linestyle='dashed', 
              linewidth=1
              )
  min_ylim, max_ylim = plt.ylim() # coleta os limites do eixo Y para definir altura dos textos
  plt.text(x.mean()*1.1,
            max_ylim*0.9,
            'Média: {:.2f}'.format(x.mean()),
            color='darkblue'
          )
  plt.text(np.median(x)*1.1,
            max_ylim*0.8, 
            'Mediana: {:.2f}'.format(np.median(x)),
            color='darkred'
          )
  # Titulo dos gráfico e dos eixos
  plt.title(subtitulo)
  plt.suptitle(titulo,fontsize=18, y=1)
  plt.xlabel(label_x)
  plt.ylabel(label_y)
  plt.show

def buscarNomes(lista):
  """Itera sob a lista de nomes

  Parameters
  ----------
  list : lista de strings

  Returns
  -------
  dictionary list:  Lista de dicionários com dados retornados
  """
  dados=[] # lista vazia
  #Itera sob a lista de nomes
  # Cada busca retorna um dicionário com os dados do autor. No fim no loop, teremos um lista de dicionários, facilmente convertido em DataFrame do Pandas
  for nome in lista:
    busca=next(scholarly.search_author(nome),False) # 1º tenta busca pelo nome completo
    if busca  != False:
      autor = scholarly.fill(busca,sections=[])
      dados.append(autor)
      # Caso não encontre pelo nome completo, criamos um "novo" nome para o autor com a função quebrarNome: 1º e o último
    elif next(scholarly.search_author(quebrarNome(nome)),False) != False:
      autor = scholarly.fill(scholarly.fill(next(scholarly.search_author(quebrarNome(nome)),False),sections=[]))
      dados.append(autor)
    else: 
      pass
  return dados


# aplica a função retorna Perfil em uma lista de strings (nomes)
def buscarPerfiz(lista):
  lista_retorno = []
  for nome in lista:
      perfil = retornaPerfil(nome)
      lista_retorno.append(perfil)
  return lista_retorno

def buscar_por_excel(tabela,inicial=0,final=0):
  tabela = pd.read_excel(tabela)
  lista = tabela.Nome
  lista = lista[inicial:final]
  dados_retorno = buscarNomes(lista)
  dados_retorno = [x for x in dados_retorno if x != False] # Remove os retornos False da lista. Sem isso o Pondas não aceita entrada dos dados
  return dados_retorno
  #print(lista)
    
def limparBol(df): 
  return [x for x in df if x != False]
    
# classe para cria objeto que usa SCRAPERAPI, uma boa ferramenta de proxy
class ScraperAPI(ProxyGenerator):
  def __init__(self, api_key):
      self._api_key = api_key
      self._client = ScraperAPIClient(api_key)
      assert api_key is not None      
      super(ScraperAPI, self).__init__()      
      self._TIMEOUT = 120
      self._session = self._client
      self._session.proxies = {}
      
  def _new_session(self):
      self.got_403 = False
      return self._session
  
  def _close_session(self):
      pass  
    
# planifica dicionário dentro do dataframe
def unpack(df, column, fillna=None):
  ret = None
  if fillna is None:
      ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
      del ret[column]
  else:
      ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
      del ret[column]
  return ret

# descarta vetores com info inúteis dentro das informações das publicacões
def drop_colunas_pubs(df):
  df=df.drop(['container_type','filled','source'], axis = 1)
  return df


df_nomes=pd.read_excel('lista de nomes.xlsx',header=None)

lista_nomes=list(df_nomes[0]) # trasforma a primeira coluna de dados em uma lista

resultado=buscarNomes(lista_nomes) # realiza a busca dos nomes e retorna uma lista de dicionários do Python. 

df=pd.DataFrame(resultado) # Lista de dicionários em dataframe do Pandas 
df.drop_duplicates(subset ="scholar_id", keep = False, inplace = True)  # Remove linhas duplicadas de acordo com scholar_id, que é unívoco

df=unpack(df, 'cites_per_year', 0) # "desempacota o dataframe dentro dessa variável em novas colunas"
df['pubs']=df['publications'].apply(pd.json_normalize) # normaliza os dados dentro da variável publications e cria nova variável
df['pubs']=df['pubs'].apply(drop_colunas_pubs) # descarta vetores com info inúteis dentro do vetor 'pubs'
df=df.drop(['container_type','filled','source','coauthors','publications'], axis = 1) # descarta vetores com info inúteis
# Salva a tabela
df.to_excel('lista de nomes.xlsx', sheet_name='Resultado da raspagem', index = False)
print('\n \n ###############################')
print('Script executado com sucesso! \n Pode baixar a planilha com os dados.')
print('\n \n ###############################')


 
 ###############################
Script executado com sucesso! 
 Pode baixar a planilha com os dados.

 
 ###############################
